<a href="https://colab.research.google.com/github/Ellen-Gu/quantity-analysis/blob/main/tf_spped_test_glab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This speed test using google colab free service (denote as G).
Other tests are done on personal laptop, one medium size public cloud (with GPU, free service, denote as M), and one small public cloud (with GPU, paid service, denote as S).

The testing code is from learning modules in google tensorflow docker image, the classification jupyter notebook. My own laptop was bought just several months earlier than the cut-off date. After that date the precision series' GPU support tensorflow 2.0 and above. Before that only tensorflow 1.X is supported. I installed tensorRT into the docker image but was very disappointed to find out that my hardware is just a little bit older than what tensorflow 2.X needed. The tesorflow 1.X codes are no longer well compatiable into tensorflow 2.X. Fortunately there are many GPU capable cloud services available outside, and tensorRT capable codes are not too far away from usual ones. At least my own laptop is merely ok for learning purpose.

One interesting point is that google tensorflow image (2.X) does not include the tensorRT from nvidia. and there is no driver from nvida can fit into the version of those used in goolge's image. If tensorRT is wanted (and openAI thinks it is a very important and "must-have" component for speeding up, without it the tensorflow can only run on CPU. Currently only nivida GPU and google's TPU are able to run tensorflow), one has to start from scratch and use cuda toolkit>12.0 (to get compatiable tensorRT from nvidia website). After tensorRT is installed, the image size may increase 20GB, that is a lot. Aside tensorRT, I guess goole's tensorflow image may be optimzied to cut off unnecessary or some optional packages so it can stay at small size. Google says tensorflow is only officially supported on ubuntu system, though one can try on other linux distributions.

From the speed testing results, if for CPU computing, my own laptop (denote as P) and the medium size public cloud (at highest speed as to free service) are with similar speed. that is, $P ≈ M$.

* CPU: $M ≈ P$, $M > S$, for spark, $M (8-threads) ≈ 1.3X \space P(4 -threads) $, $P=2X  \space G$, there is nearly no speed improvement in G's spark running.
* GPU: $M (GPU) ≈ 2X \space M(cpu) ≈ 2X  \space P(cpu)$, $S(GPU)=3X \space P(cpu)$ , and it is verified that $M$ has tensorRT installed and ready to be used. In another testing with more complex images, M's GPU classification codes can still finish in several seconds, while tensorflow 2.13 running on my laptop (can only use CPU) needs to take 1.5 hr to finish.
* TPU: G only, G(cpu) seems faster than its GPU and TPU (seems contradictary to general unstandings). Though no much speed imporovement, G is the only one which is available for testing TPU.

( $l(aX)r$ means that $l$ is $ax $ faster than $r$ )


* ###  Run tensorflow with **CPU**

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0

test_images = test_images / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)

predictions[0]
np.argmax(predictions[0])



2.12.0
4422102/4422102 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 9s 4ms/step - loss: 0.4988 - accuracy: 0.8243
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3729 - accuracy: 0.8650
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3357 - accuracy: 0.8775
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3132 - accuracy: 0.8852
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2951 - accuracy: 0.8914
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2810 - accuracy: 0.8971
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2686 - accuracy: 0.9004
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2586 - accuracy: 0.9046
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2482 - accuracy: 0.9073
Epoch 10/10


9

* ### Run tensorflow with **GPU**

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0

test_images = test_images / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)

predictions[0]
np.argmax(predictions[0])



2.12.0
4422102/4422102 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 14s 4ms/step - loss: 0.4986 - accuracy: 0.8243
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3731 - accuracy: 0.8653
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3336 - accuracy: 0.8791
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3110 - accuracy: 0.8856
Epoch 5/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.2937 - accuracy: 0.8921
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2791 - accuracy: 0.8973
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2657 - accuracy: 0.9014
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2537 - accuracy: 0.9049
Epoch 9/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2458 - accuracy: 0.9087
Epoch 10/1

9

* ### Run tensorflow with **TPU**

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0

test_images = test_images / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)

predictions[0]
np.argmax(predictions[0])



2.12.0
4422102/4422102 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 16s 7ms/step - loss: 0.5008 - accuracy: 0.8230
Epoch 2/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.3793 - accuracy: 0.8634
Epoch 3/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.3406 - accuracy: 0.8762
Epoch 4/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.3162 - accuracy: 0.8844
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2956 - accuracy: 0.8912
Epoch 6/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2810 - accuracy: 0.8957
Epoch 7/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2675 - accuracy: 0.9008
Epoch 8/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2562 - accuracy: 0.9045
Epoch 9/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2469 - accuracy: 0.9082
Ep

9